In [1]:
import sys
sys.path.append("../..")
from time import sleep
from common import *
from ad9174 import Ad9174Settings, Ad9174Init

In [81]:
settings = Ad9174Settings(json_file='../build/csr.json')
print(settings)

r = conLitexServer('../build/csr.csv')

ad = Ad9174Init(r, settings)

----------------
 JESD mode 2
----------------
INTERP_CH: 4  INTERP_MAIN: 8  DSP_CLK_DIV: 32
JESD204BSettings(): 5a 05 00 82 03 1f 05 0f 2f 20 80 00 00 e6 
         DID:  90        BID:   5     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   3        SCR:   1 
           F:   4          K:  32          M:   6          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   1 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 230 
   [ LINK_DW:  32     FR_CLK:   1 ]
Connected to Port 1234
AD9174 + VC707 test 2020-04-27 23:27:19


# 1. init AD9174

In [82]:
ad.init_ad9174()

AD917X_NVM_BLR_DONE: 1
PROD_ID: 0x9174
PROD_GRADE: 0  DEV_REVISION: 5
DLL locked: 1
SPI_PAGEINDX: 0b01000001
CAL_STAT: 1
SERDES PLL locked: 1
MODE_NOT_IN_TABLE: 0


# 2. init HMC7044

In [83]:
ad.init_hmc()
# ad.hmc.trigger_div_reset()

In [85]:
ad.fpga_print_clocks()

f_jesd = 160.000444 MHz  f_ref = 0.500001 MHz


# 3. Init the FPGA side

In [86]:
# r.regs.ctrl_reset.write(1)  # resets ALL clockdomains (HARSH!)
r.regs.control_control.write(0b01)  # resets PHYs and jesd core
print('status: {:03b}'.format(r.regs.control_status.read()))

# bit1: links_enable,  bit0: phys_reset
r.regs.control_control.write(0b10)
# bit2: /jsync,  bit1: links_ready,  bit0: phys_ready
print('status: {:03b}'.format(r.regs.control_status.read()))

ad.trigger_jref_sync()
ad.print_irq_flags(True, True)
if ad.print_irq_flags(True):
    print('😭')
else:
    print('😃')

status: 000
status: 111
SYNC_ROTATION_DONE 1
DYN_LINK_LATENCY 15 cycles
😃


# 4. Test link

In [87]:
ad.print_irq_flags(True)
print()

ad.print_ilas()

ad.print_lane_status()
print()

ad.test_stpl()


JESD settings, received on lane 0 vs (programmed):
450: 5a (5a)
451: 05 (05)
452: 00 (00)
453: 82 (82)
454: 03 (03)
455: 1f (1f)
456: 05 (05)
457: 0f (0f)
458: 2f (2f)
459: 20 (20)
45a: 80 (80)
45b: 00 (00)
45c: 00 (00)
45d: e6 (e6)
CHK: e6 (e6) 

Lane status:
      LANE_DESKEW: 00000111
    BAD_DISPARITY: 00000000
     NOT_IN_TABLE: 00000000
 UNEXPECTED_KCHAR: 00000000
    CODE_GRP_SYNC: 00000111
       FRAME_SYNC: 00000111
    GOOD_CHECKSUM: 00000111
   INIT_LANE_SYNC: 00000111
FIFO_STATUS_REG_0: 00000000
FIFO_STATUS_REG_1: 00000000
fpga j_sync errs: 2

STPL test:
converter: 0, sample: 0, tp: 597a, fail: 0
converter: 1, sample: 0, tp: d27a, fail: 0
converter: 2, sample: 0, tp: 4b7a, fail: 0
converter: 3, sample: 0, tp: c47a, fail: 0
converter: 4, sample: 0, tp: 3d7a, fail: 0
converter: 5, sample: 0, tp: b67a, fail: 0


0

# app layer PRBS
gateware work in progress. Generating multiple samples / clock is hard :p

In [88]:
r.regs.prbs_gen_sample_prbs_en.write(1)

In [89]:
r.regs.prbs_gen_sample_prbs_en.read()

1

In [79]:
ad.ad.help(0x14b)

reg 0x14b, PRBS:
    bit 7, PRBS_GOOD_Q, R, reset 0x00
    DAC1 good data indicator. 1: Correct PRBS sequence detected. 0: Incorrect sequence detected. Sticky; reset to 1 by PRBS_RESET.

    bit 6, PRBS_GOOD_I, R, reset 0x00
    DAC0 good data indicator. 0: Incorrect sequence detected. Sticky; reset to 1 by PRBS_RESET. 1: Correct PRBS sequence detected.

    bit 5, RESERVED, R, reset 0x00
    Reserved.

    bit 4, PRBS_INV_Q, R/W, reset 0x01
    DAC1 data inversion. 0: Expect normal data. 1: Expect inverted data.

    bit 3, PRBS_INV_I, R/W, reset 0x00
    DAC0 data inversion. 0: Expect normal data. 1: Expect inverted data.

    bit 2, PRBS_MODE, R/W, reset 0x00
    Select which PRBS polynomial is used for the datapath PRBS test. 0: 7-bit: x7 + x6 + 1. 1: 15-bit: x15 + x14 + 1.

    bit 1, PRBS_RESET, R/W, reset 0x00
    Reset error counters. 0: Normal operation. 1: Reset counters.

    bit 0, PRBS_EN, R/W, reset 0x00
    Enable PRBS checker. 0: Disable. 1: Enable.



In [94]:
ad.ad.wr(0x14b, 0b0111)
ad.ad.wr(0x14b, 0b0101)

for ch in range(7):
    ad.ad.wr(0x14e, ch)
    print('{:d}: {:08b}, {:3d}, {:3d}'.format(ch, ad.ad.rr(0x14b), ad.ad.rr(0x14c), ad.ad.rr(0x14d)))

0: 00000101, 255, 255
1: 00000101, 255, 255
2: 00000101, 255, 255
3: 11000101,   0,   0
4: 11000101,   0,   0
5: 11000101,   0,   0
6: 00000101, 255, 255
